In [0]:
# !pip install --upgrade tables
# !pip install eli5
# !pip install xgboost

In [0]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [4]:
df = pd.read_hdf("data/car.h5")
df.shape

(106494, 155)

In [5]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

# **Feature Engineering**


In [6]:
df = df[df['price_currency'] != 'EUR']
df.shape

(106290, 155)

In [0]:
SUFFIX_CAT = '__cat'


for feat in df.columns:
  if isinstance(df[feat][0],list): continue

  factorized_values = df[feat].factorize()[0]

  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [8]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]

len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [10]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19566.588937368328, 90.61814865166)

In [11]:
model = RandomForestRegressor(max_depth=5, n_estimators=10,random_state=0)
run_model(model,cat_feats)

(-18786.163478267277, 174.7193804859367)

##XGBoost

In [12]:
xgb_params = {
    'max_depth' : 5,
    'n_estimators' : 50,
    'learning_rate' : 0.1,
    'seed' : 0
}


run_model(xgb.XGBRegressor(**xgb_params),cat_feats)

[23:37:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:38:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:38:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13039.290196724838, 109.36715375706265)

In [14]:
m = xgb.XGBRegressor(max_depth=5,n_estimators=50,learning_rate=0.1,seed=0)
X = df[cat_feats].values
y= df['price_value'].values
m.fit(X,y)

imp = PermutationImportance(m,random_state=0).fit(X,y)
eli5.show_weights(imp,feature_names=cat_feats)

[23:40:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1209 ± 0.0019,param_napęd__cat
0.1175 ± 0.0030,param_rok-produkcji__cat
0.1113 ± 0.0013,param_stan__cat
0.0625 ± 0.0019,param_skrzynia-biegów__cat
0.0527 ± 0.0016,param_faktura-vat__cat
0.0461 ± 0.0015,param_moc__cat
0.0275 ± 0.0008,param_marka-pojazdu__cat
0.0230 ± 0.0004,param_typ__cat
0.0227 ± 0.0007,feature_kamera-cofania__cat
0.0191 ± 0.0007,param_pojemność-skokowa__cat


In [16]:
feats = ['param_napęd__cat',
'param_rok-produkcji__cat',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc__cat',
'param_marka-pojazdu__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'param_kod-silnika__cat',
'param_model-pojazdu__cat',
'feature_wspomaganie-kierownicy__cat',
'param_wersja__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_regulowane-zawieszenie__cat',
'feature_system-start-stop__cat',
'feature_światła-led__cat'] 
len(feats)

20

In [17]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) =='None' else int(x) )
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat',
'param_moc__cat','param_marka-pojazdu__cat','param_typ__cat','feature_kamera-cofania__cat','param_pojemność-skokowa__cat',
'seller_name__cat','param_kod-silnika__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','param_wersja__cat',
'feature_czujniki-parkowania-przednie__cat','feature_asystent-pasa-ruchu__cat','feature_regulowane-zawieszenie__cat',
'feature_system-start-stop__cat','feature_światła-led__cat']


run_model(xgb.XGBRegressor(**xgb_params),feats)

[23:48:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:48:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:48:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11197.83713694348, 98.22041147876314)

In [18]:
df['param_moc'] = df['param_moc'].map(lambda x : -1 if str(x) =='None' else int(x.split('KM')[0].replace(' ','')))
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat',
'param_moc','param_marka-pojazdu__cat','param_typ__cat','feature_kamera-cofania__cat','param_pojemność-skokowa__cat',
'seller_name__cat','param_kod-silnika__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','param_wersja__cat',
'feature_czujniki-parkowania-przednie__cat','feature_asystent-pasa-ruchu__cat','feature_regulowane-zawieszenie__cat',
'feature_system-start-stop__cat','feature_światła-led__cat']

run_model(xgb.XGBRegressor(**xgb_params),feats)

[23:48:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:48:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:48:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9604.134332911071, 52.171731032910024)

In [19]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x : -1 if str(x) =='None' else int(x.split('cm')[0].replace(' ','')))

feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat',
'param_moc','param_marka-pojazdu__cat','param_typ__cat','feature_kamera-cofania__cat','param_pojemność-skokowa',
'seller_name__cat','param_kod-silnika__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','param_wersja__cat',
'feature_czujniki-parkowania-przednie__cat','feature_asystent-pasa-ruchu__cat','feature_regulowane-zawieszenie__cat',
'feature_system-start-stop__cat','feature_światła-led__cat']

run_model(xgb.XGBRegressor(**xgb_params),feats)

[23:49:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:49:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:49:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9467.559541184863, 106.52529025706468)